# Environment

In [2]:
import numpy as np, pandas as pd, tensorflow as tf

from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession

# Hyperparameters

In [3]:
BUFFER_SIZE = 100
BATCH_SIZE = 32
EPOCHS = 10

# BigQuery Connector

Documentation: https://github.com/tensorflow/io/tree/master/tensorflow_io/bigquery

In [4]:
GCP_PROJECT_ID = "axa-ch-machine-learning-dev"
DATASET_GCP_PROJECT_ID = "axa-ch-machine-learning-dev"
DATASET_ID = "nlp_text_classification"
TABLE_ID = "stackoverflow_posts"

In [5]:
client = BigQueryClient()

read_session = client.read_session(
    "projects/" + GCP_PROJECT_ID,
    DATASET_GCP_PROJECT_ID,
    TABLE_ID,
    DATASET_ID,
    ["body", "tags"],
    [dtypes.string, dtypes.string],
    row_restriction="",
)
dataset = (
    read_session.parallel_read_rows()
    .shuffle(buffer_size=BUFFER_SIZE)
    .batch(batch_size=BATCH_SIZE)
)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


A tf.data dataset. Should return a tuple of either (inputs, targets) or (inputs, targets, sample_weights). <br>
Problem: the dataset above does not return a tuple of unputs and labels

# Feature Layer

In [1]:
feature_names = ['title', 'body', 'tags']

In [2]:
feature_columns = []

for name in feature_names:
    feature_columns.append(tf.feature_column.numeric_column(name))

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

NameError: name 'tf' is not defined

# Model

In [ ]:
model = tf.keras.Sequential(
    [
        feature_layer,
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Parameters: <a href src='https://www.tensorflow.org/api_docs/python/tf/keras/Model' target='_blank'>model.fit</a>

# Fit Model (Non-Dataset Input)

In [ ]:
model.fit(
    x=None,
    y=None,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=2,
    validation_split=0.2,
    shuffle=True,
)

# Fit Model (Dataset Input)

In [ ]:
model.fit(dataset, verbose=2)

# Evaluate Model

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

https://towardsdatascience.com/multi-label-image-classification-with-neural-network-keras-ddc1ab1afede